In [23]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [25]:
data = pd.read_csv("monte_carlo.csv")

In [22]:
class NashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        p = np.zeros((2,2,n))

        for a_j in [0,1]:
            util1 = np.dot(self.exog[:,0:k], params[0:k])  + params[k]*a_j
            util2 = np.dot(self.exog[:,k:], params[k+1:2*k+1]) + params[2*k+1]*a_j
            p[0,a_j,:] = 1 /(1 + np.exp(util1))
            p[1,a_j,:] = 1 /(1 + np.exp(util2))
        
        lamb = np.exp(params[-1])/(1+np.exp(params[-1]))
        
        #solve for probablity of nash
        mult_eq = np.maximum( ( p[0,1,:]  - p[0,0,:] )*(  p[1,1,:] - p[1,0,:] ),0)
        prob01 = ( p[0,1,:] )*( 1 - p[1,0,:] ) - (1-lamb)*mult_eq
        prob10 = ( 1 - p[0,0,:] )*( p[1,1,:] ) - (lamb)*mult_eq
        prob00 = p[0,0,:] * p[1,0,:]
        prob11 = ( 1 - p[0,1,:] )*( 1 - p[1,1,:] )

        #compute empirical likelihood
        p00 = (1 - self.endog[:,0])*(1 - self.endog[:,1])
        p11 = self.endog[:,0]*self.endog[:,1]
        p10 = self.endog[:,0]* (1 - self.endog[:,1])
        p01 = (1 - self.endog[:,0]) * self.endog[:,1]

        ll = p00 * prob00 + p11 *prob11 + p01 * prob01 + p10 * prob10
        
       
        return -np.log(ll).sum()
        
        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+3)
        start_params = np.array([1, 4, -3 ,2, 3, -2 ,-1])
        return super(NashLogit, self).fit(start_params=start_params,**kwds)

        
        
N = data.shape[0]
y = data[['y1','y2']]
x = data[['x1','x2']]

x.insert(0,'x01',np.ones(N))
x.insert(2,'x02',np.ones(N))

model = NashLogit(y,x)
model = model.fit()
print(model.summary(xname=["b01","b11","d1","b02","b12","d2","rho"]))

                              NashLogit Results                               
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -576.50
Model:                      NashLogit   AIC:                             1159.
Method:            Maximum Likelihood   BIC:                             1174.
Date:                Fri, 05 Mar 2021                                         
Time:                        14:20:31                                         
No. Observations:                1000                                         
Df Residuals:                     997                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
b01            0.9972      0.188      5.312      0.000       0.629       1.365
b11            4.1762      0.298     14.008      0.0

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
